<a href="https://colab.research.google.com/github/OlgaPriyutova/for-General-Assembly/blob/main/GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

In [136]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score






In [137]:
headers = open('part-1-field-names.txt')
col =headers.readlines()
colu = [k.rstrip('\n') for k in col]

In [138]:
file = pd.read_csv('part-1-data.csv',header=None)
file.columns = colu
file.head()


,ID,diagnosis,radius_mean,radius_sd_error,radius_worst,texture_mean,texture_sd_error,texture_worst,perimeter_mean,perimeter_sd_error,...,concavity_worst,concave_points_mean,concave_points_sd_error,concave_points_worst,symmetry_mean,symmetry_sd_error,symmetry_worst,fractal_dimension_mean,fractal_dimension_sd_error,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Check for the missing values:

In [139]:
file.isna().sum()

ID                            0
diagnosis                     0
radius_mean                   0
radius_sd_error               0
radius_worst                  0
texture_mean                  0
texture_sd_error              0
texture_worst                 0
perimeter_mean                0
perimeter_sd_error            0
perimeter_worst               0
area_mean                     0
area_sd_error                 0
area_worst                    0
smoothness_mean               0
smoothness_sd_error           0
smoothness_worst              0
compactness_mean              0
compactness_sd_error          0
compactness_worst             0
concavity_mean                0
concavity_sd_error            0
concavity_worst               0
concave_points_mean           0
concave_points_sd_error       0
concave_points_worst          0
symmetry_mean                 0
symmetry_sd_error             0
symmetry_worst                0
fractal_dimension_mean        0
fractal_dimension_sd_error    0
fractal_

In [140]:
file.loc[file['diagnosis'] == 'M', 'cancer_present'] = 1
file.loc[file['diagnosis'] == 'B', 'cancer_present'] = 0
file.columns


Index(['ID', 'diagnosis', 'radius_mean', 'radius_sd_error', 'radius_worst',
       'texture_mean', 'texture_sd_error', 'texture_worst', 'perimeter_mean',
       'perimeter_sd_error', 'perimeter_worst', 'area_mean', 'area_sd_error',
       'area_worst', 'smoothness_mean', 'smoothness_sd_error',
       'smoothness_worst', 'compactness_mean', 'compactness_sd_error',
       'compactness_worst', 'concavity_mean', 'concavity_sd_error',
       'concavity_worst', 'concave_points_mean', 'concave_points_sd_error',
       'concave_points_worst', 'symmetry_mean', 'symmetry_sd_error',
       'symmetry_worst', 'fractal_dimension_mean',
       'fractal_dimension_sd_error', 'fractal_dimension_worst',
       'cancer_present'],
      dtype='object')

# Median smoothness: Compute the median smoothness for benign and malignant tumors.

In [141]:
print('Median smootheness for malignant = ',file[file['diagnosis']=='M']['smoothness_mean'].median())
print('Median smootheness for benign = ',file[file['diagnosis']=='B']['smoothness_mean'].median())



Median smootheness for malignant =  3.6795
Median smootheness for benign =  1.851


Let us choose features and target columns  (RFE estimator will help us to choose main features itself), and split the data for the train and test datasets.

In [142]:
y = file['cancer_present']
X = file.drop(['ID','diagnosis','cancer_present'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

# Application of Logistic Regression: 

In [143]:
logreg    = Pipeline([('scaler', StandardScaler()), 
                     ('rfe', RFE(estimator=LogisticRegression(), n_features_to_select=5)),
                     ('log_reg', LogisticRegression(random_state=32))])

logreg.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rfe',
                 RFE(estimator=LogisticRegression(), n_features_to_select=5)),
                ('log_reg', LogisticRegression(random_state=32))])

In [144]:
cross_val_score(logreg, X_train, y_train, cv=10).mean()

0.9625692137320044

# Application of Naive Baiess Algorithm:

In [145]:
nb     = Pipeline([('scaler', StandardScaler()), ('nb', GaussianNB())])

nb.fit(X_train, y_train)

cross_val_score(nb, X_train, y_train, cv=10).mean()

0.9343300110741971

# Application of Gradient Booster Algorithm:

In [146]:
GradientBoost     = Pipeline([('scaler', StandardScaler()), 
                     ('Gradient', GradientBoostingClassifier(random_state=32))])

GradientBoost.fit(X_train, y_train)

cross_val_score(GradientBoost, X_train, y_train, cv=10).mean()

0.9601882613510521

# Comparing accuracy for each algorithm: 

In [147]:



print(cross_val_score(logreg, X_train, y_train, cv=5).mean())
print(cross_val_score(GradientBoost, X_train, y_train, cv=5).mean())
print(cross_val_score(nb, X_train, y_train, cv=5).mean())

0.955403556771546
0.9601094391244871
0.9389603283173734


# Prediction on test data by all 3 algorithms: 

In [148]:
pl = logreg.predict(X_test)
pgb = GradientBoost.predict(X_test)
pnb = nb.predict(X_test)

print(precision_score(y_test, pl))
print(precision_score(y_test, pgb))  ## Prediction on test data GradientBuster algorithm gave us better results
print(precision_score(y_test, pnb))

0.9016393442622951
0.9454545454545454
0.9090909090909091


#Gradient Bosster algorithm gave us best result out of 3 algorithm - Logistic Regression. Naive Baiess and GB. 

Next steps: we should check how Decision Tree and Random Forest Algorithms will work on this data. 






## Part 2

In [149]:

#Student 2

#import pandas as pd
#import numpy as np
#from sklearn.linear_model import LinearRegression
#from sklearn.cross_validation import cross_val_score  

#### There CrossValidations comes form sklearn.model_selection module

# Load data
#data = pd.read_csv('../data/train.csv') # It would be nice to see the full path


# Setup data for prediction
#y = data.SalaryNormalized
#X = pd.get_dummies(data.ContractType)

#### Rem Unfortunately, you haven't use train-test split. 
# Setup model
#model = LinearRegression()
#### REM Model should be trained using FIT command!!!   
# Evaluate model
#scores = cross_val_score(model, X, y, cv=5, scoring='mean_absolute_error')
#print(scores.mean())

In [150]:

# Student 1 

#import pandas as pd
#import numpy as np
#from sklearn import LinearRegression
#from sklearn.cross_validation import cross_val_score

## REM LinearRegression comes form sklearn.linear_model  
## REM from sklearn.model_selection import cross_val_score  
##
# Load data
#d = pd.read_csv('../data/train.csv')  # It would be nice to see the full path


# Setup data for prediction
#x1 = data.SalaryNormalized

##  REM The name of your dateframe is d, not data. You chose the name youself.     
## REM   X, y - standard notation for features and the target column, not x1 and x2.
#x2 = pd.get_dummies(data.ContractType)

# Setup model
#model = LinearRegression()
## REM    THE model only initialized, but not trained using FIT command


## REM All library are supposed to be mentioned at the top.  
##   from sklearn.model_selection import cross_val_score
## - how the CrossValidation score is supposed to be imported
## Usage of Cross Validation makes sense if cv >1, otherwise there are other accuracy measures available. 
## Unfortunately, you haven't use train-test split. 
## from sklearn.model_selection import train_test_split  - both CVs and TTsptill come from ModelSelection module

#from sklearn.cross_validation import cross_val_score
#from sklearn.cross_validation import train_test_split
#scores = cross_val_score(model, x2, x1, cv=1, scoring='mean_absolute_error')
#print(scores.mean())